# Mosaico 2021 C2

Script trabajado por Francisco Albornoz/ 
Objetivo:   - Mejorar la temporalidad de los mosaicos.
            - Usar la colección corregida.
            

2022-03-31

## Librerías

In [ ]:
import ee 
import geemap
import geemap.chart as chart
import os
import folium
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr
%matplotlib inline

In [ ]:
Map = geemap.Map(center = [-45, -70], zoom = 4)
Map1 = geemap.Map(center = [-45, -70], zoom = 4)
Map2 = geemap.Map(center = [-45, -70], zoom = 4)
Map3 = geemap.Map(center = [-45, -70], zoom = 4)
Map4 = geemap.Map(center = [-45, -70], zoom = 4)
Map5 = geemap.Map(center = [-45, -70], zoom = 4)
Map6 = geemap.Map(center = [-45, -70], zoom = 4)
Map7 = geemap.Map(center = [-45, -70], zoom = 4)
Map8 = geemap.Map(center = [-45, -70], zoom = 4)
Map9 = geemap.Map(center = [-45, -70], zoom = 4)
Map10 = geemap.Map(center = [-45, -70], zoom = 4)

## Área de Estudio

In [ ]:
work_dir = os.path.expanduser('../Data/')
patagonia = os.path.join(work_dir, 'patagonia_geo.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

## Funciones

# Image Collections